In [ ]:
!unrar x '/content/drive/My Drive/project/mango/train_dev/C2_TrainDev.rar' '/content/drive/My Drive/project/train_dev/'

In [ ]:
import cv2
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
csv_path = '/content/drive/My Drive/project/mango/sample/label'
image_path = '/content/drive/My Drive/project/mango/sample/image/'

# csv_path = '/content/drive/My Drive/project/mango/train/label'
# image_path = '/content/drive/My Drive/project/mango/train/image/'

# csv_path = '/content/drive/My Drive/project/mango/dev/label'
# image_path = '/content/drive/My Drive/project/mango/dev/image/'

# csv_path = '/content/drive/My Drive/project/mango/test/'
# image_path = '/content/drive/My Drive/project/mango/test/image/Test/'

In [ ]:
def load_csv(csv_path, file_name):
    # dataset = pd.read_csv(os.path.join(csv_path, file_name), low_memory = False)
    dataset = pd.read_csv(os.path.join(csv_path, file_name), header = None, low_memory = False)
    dataset.fillna('1', inplace = True)
    dataset_array = np.array(dataset)
    return dataset, dataset_array

In [ ]:
dataset, dataset_array = load_csv(csv_path, 'label.csv')
# dataset, dataset_array = load_csv(csv_path, 'train.csv')
# dataset, dataset_array = load_csv(csv_path, 'dev.csv')
# dataset, dataset_array = load_csv(csv_path, 'Test_mangoXYWH.csv')
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,00117.jpg,不良-著色不佳,340,135,363,252,1,1,1,1,1
1,00222.jpg,不良-著色不佳,309,199,396,408,1,1,1,1,1
2,00349.jpg,不良-著色不佳,330,139,297,391,1,1,1,1,1
3,00595.jpg,不良-著色不佳,347,131,473,484,1,1,1,1,1
4,00860.jpg,不良-著色不佳,221,180,277,282,不良-著色不佳,599,251,137,312


# Write TFRecord Dataset

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

def segment(data):
    feature_set= []
    img_name = []
    demension = len(data[0])
    segment = list(range(1, demension, 5))
    for i in range(len(data)):
        for seg in segment:
            if data[i][seg] == '1':
                pass
            else:
                sub_data = data[i][seg:seg + 5]
                img = data[i][0]
                feature_set.append(sub_data)
                img_name.append(img)
    img_name = np.array(img_name)
    feature_set = np.array(feature_set)
    label = feature_set[:,4]
    w_h = feature_set[:,:4].astype('float32')
    return img_name, feature_set, label, w_h


def image_example(image, category):
    # image = plt.imread(os.path.join(image_path, file_name))
    # image_string = open(os.path.join(image_path, file_name), 'rb').read()
    # image_shape = tf.image.decode_jpeg(image_string).shape
    image_string = tf.image.encode_jpeg(image)
    feature = {
      'image_raw': _bytes_feature(image_string),
      'target':_bytes_feature(category),
    }
    return tf.train.Example(features = tf.train.Features(feature = feature))

def image_example_2(image, cordinate):
    image_string = tf.image.encode_jpeg(image)
    feature = {
      'image_raw': _bytes_feature(image_string),
      'cordinate' : _bytes_feature(cordinate)
    }
    return tf.train.Example(features = tf.train.Features(feature = feature))
def y_label(label):
  target = np.zeros((5,4), dtype = 'float32')
  if label == '不良-乳汁吸附':
    o_h_e[0,0] = 1
  elif label == '不良-機械傷害':
    o_h_e[0,1] = 1
  elif label == '不良-炭疽病':
    o_h_e[0,2] = 1
  elif label == '不良-著色不佳':
    o_h_e[0,3] = 1
  else:
    o_h_e[0,4] = 1
  # return o_h_e.tostring()
  return o_h_e.tobytes()
  # return o_h_e

In [ ]:
np.zeros((5,4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
np.zeros((4,5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

#Read TF Dataset

In [ ]:
BATCH_SIZE = 100
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = image/255
    # image = tf.reshape(image, [224, 224, 3])
    return image

def decode_label(label):
  label = tf.io.decode_raw(label, tf.float32)
  # label = p
  # label = tf.cast(label, tf.float32)
  return label

def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.string),
            'cordinate': tf.io.FixedLenFeature([], tf.string)
        }
        if labeled
        else {"image_raw": tf.io.FixedLenFeature([], tf.string)}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image_raw'])
    if labeled:
        label = decode_label(example['target'])
        cor = decode_label(example['cordinate'])
        # label = tf.cast(label, tf.int32)
        return image, label, cor
    return image

def load_dataset(filenames, labeled = False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = True  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  
# uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled = labeled), num_parallel_calls = AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled = False):
    dataset = load_dataset(filenames, labeled = labeled)
#     dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size = AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

# Sample

In [ ]:
def segment_2(data):
    feature_set= []
    img_name = []
    demension = len(data[0])
    segment = list(range(1, demension, 5))
    for i in range(len(data)):
        for seg in segment:
            if data[i][seg] == '1':
                pass
            else:
                sub_data = data[i][seg:seg + 5]
                img = data[i][0]
                feature_set.append(sub_data)
                img_name.append(img)
    img_name = np.array(img_name)
    feature_set = np.array(feature_set)
    label = feature_set[:,0]
    w_h = feature_set[:,1:].astype('float32')
    return img_name, feature_set, label, w_h

In [ ]:
img_name, feature_set, label, w_h = segment_2(dataset_array)

In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'sample.tfrecord')) as writer:
  i, j = 0, 0
  for image in img_name:
    cordinate = feature_set[i, 1:].astype(np.float32)
    cordinate = cordinate.reshape((1, 4))
    b_cordinate = cordinate.tobytes()
    category = one_hot(feature_set[i,0])
    if os.path.isfile(image_path + image):
      # print(image + ' was read')
      w_image = plt.imread(os.path.join(image_path, image))
      w_image = cv2.resize(w_image, (224,224))
      tf_example = image_example(w_image, category, b_cordinate)
      writer.write(tf_example.SerializeToString())
      j += 1
    i += 1
writer.close() 
print(i)
print(j)

107
107


# Train

In [ ]:
img_name, feature_set, label, w_h = segment(dataset_array)

In [ ]:
print(np.concatenate((category, cordinate), axis = 1).shape)

(1, 9)


In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'train.tfrecord')) as writer:
  i, j = 0, 0
  for image in img_name:
    cordinate = feature_set[i,:4].astype(np.float32)
    cordinate = cordinate.reshape((1, 4))
    category = one_hot(feature_set[i,4])
    total = np.concatenate((category, cordinate), axis = 1)
    total = total.tobytes()
    if os.path.isfile(image_path + image):
      # print(image + ' was read')
      w_image = plt.imread(os.path.join(image_path, image))
      w_image = cv2.resize(w_image, (224,224))
      tf_example = image_example(w_image, total)
      writer.write(tf_example.SerializeToString())
      j += 1
    i += 1
writer.close() 
print(i)
print(j)

In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'train.tfrecord')) as writer:
i, j = 0, 0
  for image in img_name:
    k = 0
    cordinate = feature_set[i,:4]
    category = one_hot(feature_set[i,4])
    # print(category)
    # print('---------------------------------------------------')
    cordinate = cordinate.astype('int32')
    if os.path.isfile(image_path + image):
      # print(image + ' was read')
      w_image = plt.imread(os.path.join(image_path, image))
      crop_image = w_image[y:y+h, x:x+w]
      crop_image = cv2.resize(crop_image, (224,224))
      for batch in image_gen_train.flow(img, batch_size = 32):
        augImage = batch[0]
        augImage = augImage.astype('float32')
        augImage = cv2.resize(augImage, (224, 224))
        tf_example = image_example(augImage, category)
        writer.write(tf_example.SerializeToString())
        k += 1
        if k >= 20:
          break
      j += 1
    i += 1
writer.close() 
print(i)
print(j)

43370
43370


#Dev

In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'dev.tfrecord')) as writer:
  i, j = 0, 0
  for image in img_name:
    cordinate = feature_set[i,:4].astype(np.float32)
    cordinate = cordinate.reshape((1, 4))
    category = one_hot(feature_set[i,4])
    total = np.concatenate((category, cordinate), axis = 1)
    total = total.tobytes()
    if os.path.isfile(image_path + image):
      # print(image + ' was read')
      w_image = plt.imread(os.path.join(image_path, image))
      w_image = cv2.resize(w_image, (224,224))
      tf_example = image_example(w_image, total)
      writer.write(tf_example.SerializeToString())
      j += 1
    i += 1
writer.close() 
print(i)
print(j)

In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'dev.tfrecord')) as writer:
  i = 0
  j = 0
  for image in img_name:
    cordinate = feature_set[i,:4]
    category = one_hot(feature_set[i,4])
    # print(category)
    # print('---------------------------------------------------')
    cordinate = cordinate.astype('int32')
    x,y,w,h = cordinate
    if os.path.isfile(image_path + image):
      # print(image + ' was read')
      w_image = plt.imread(os.path.join(image_path, image))
      crop_image = w_image[y:y+h, x:x+w]
      crop_image = cv2.resize(crop_image, (224,224))
      tf_example = image_example(crop_image, w, h, category)
      writer.write(tf_example.SerializeToString())
      j += 1
    i += 1
writer.close() 
print(i)
print(j)

43370
43370


#Test

In [ ]:
cor = dataset_array[:, 1:]
with tf.io.TFRecordWriter(os.path.join(csv_path, 'test.tfrecord')) as writer:
  a, b = 0, 0
  for i in range(len(dataset_array)):
    x, y, w, h = cor[i]
    img_name = dataset_array[i, 0]
    if os.path.isfile(image_path + img_name):
      w_image = plt.imread(os.path.join(image_path, img_name))
      crop_image = w_image[y:y+h, x:x+w]
      crop_image = cv2.resize(crop_image, (224, 224))
      tf_example = image_example_2(crop_image)
      writer.write(tf_example.SerializeToString())
      b += 1
    a += 1
writer.close()
print(a, b)

7363 7363
